In [ ]:
#Author: ML Tlachac
#Paper: Depression Screening from Text Message Reply Latency
#year: 2020
#github.com/mltlachac/EMBC2020

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF


#randoms = []
#for r in range(0, 100):
#    randoms.append(random.randint(1,1000000000))
#print(randoms)

In [ ]:
randoms = [361128390, 25678624, 286227372, 275413183, 835799483, 650031229, 381128932, 444496640, 170857706, 432939712, 369429758, 115346405, 35890687, 309457986, 109332894, 953403852, 252819303, 738792529, 241761773, 310165146, 896869903, 93132008, 144816658, 191300349, 602971087, 512359958, 399426044, 749247493, 414568457, 346244712, 736079420, 273441439, 142976366, 109103084, 925228403, 107949476, 738918921, 203448980, 922033929, 655505189, 48433776, 649673706, 321823331, 866307439, 217716963, 593539541, 803328973, 384847421, 31408796, 87997717, 592527218, 506246877, 590053539, 736896836, 252708346, 313920490, 623320593, 958675774, 364213774, 98505089, 59689233, 442316781, 326740111, 958938896, 759344068, 508584824, 645914056, 969919596, 969189324, 496406392, 660520068, 144918304, 653747834, 167562051, 900122744, 135334227, 966440225, 973306951, 495640847, 195133324, 496810208, 353187995, 814586150, 238549354, 419037670, 709650294, 549538988, 666493106, 606904926, 450619297, 675703044, 36759020, 102404976, 561079479, 670036183, 872070941, 909164647, 169045909, 730926233, 136792799]

In [ ]:
scoring = ['precision', 'recall', 'f1','accuracy', 'roc_auc']
featureElist = ["Chi", "PCA", "kPCA"]
splitlist = [10, 12, 15]

namelist = ["Texts"]
ndayslist = [14, 28, 42, 364]
modelTypelist = ["NB", "LR", "SVC1", "SVC2", "kNN3", "kNN5", "XG", "RF"]

for split in splitlist:
    for featureE in featureElist:
        for name in namelist:
            for ndays in ndayslist:
                for modelType in modelTypelist:

                    df = pd.read_csv("latency" + name + str(ndays) + "cleaned.csv")
                    cnames = df.columns

                    df0 = pd.DataFrame()
                    for c in cnames:
                        df0[c] = df[c].fillna(0)

                    data = df0
                    featureDF = []
                    data[data.columns[-1]] = np.where(data[data.columns[-1]] >= split, 1, 0)
                    featureSubset = data[data.columns[1:-1]]
                    target = data[data.columns[-1]]
                    min_max_scaler = preprocessing.MinMaxScaler()                   #NEED TO SCALE BEFORE FEATURE SELECTION!
                    np_scaled = min_max_scaler.fit_transform(featureSubset)
                    featureSubset = pd.DataFrame(np_scaled)

                    if featureE == "Chi":
                        nFeatureList = list(np.arange(1,11,1))

                        for numberOfFeatures in nFeatureList:
                            featureSubset2 = SelectKBest(chi2, k=numberOfFeatures).fit_transform(featureSubset, target)
                            featureSubset2=pd.DataFrame(featureSubset2).assign(target = target)
                            featureDF.append(featureSubset2)

                    elif featureE == "kPCA":
                        nFeatureList = list(np.arange(1,6,1))

                        featureSubset2 = featureSubset#SelectKBest(chi2, k=17).fit_transform(featureSubset, target) #or feature subset
                        for numberOfFeatures in nFeatureList:
                            pca = KernelPCA(n_components=numberOfFeatures, kernel = "rbf")
                            pca.fit(featureSubset2)
                            X_pca = pca.transform(featureSubset2)
                            pcaDF = pd.DataFrame(X_pca)#pca.components_.transpose())
                            pcaDF = pcaDF.assign(target = target)
                            

                            featureDF.append(pcaDF)

                    elif featureE == "PCA":
                        nFeatureList = list(np.arange(1,6,1))

                        featureSubset2 = featureSubset#SelectKBest(chi2, k=17).fit_transform(featureSubset, target) #or feature subset
                        for numberOfFeatures in nFeatureList:
                            pca = PCA(n_components=numberOfFeatures)
                            pca.fit(featureSubset2)
                            X_pca = pca.transform(featureSubset2)
                            pcaDF = pd.DataFrame(X_pca)#pca.components_.transpose())
                            pcaDF = pcaDF.assign(target = target)
                            

                            featureDF.append(pcaDF)


                    alist = []
                    flist = []
                    mlist = []
                    plist = []
                    featureList = []
                    precisionList = []
                    recallList = []
                    f1List = []
                    accuracyList = []
                    aucList = []
                    truePosList = []
                    trueNegList = []
                    falsePosList = []
                    falseNegList = []
                    predictions = []
                    randomseed = []
                    shape = []

                    for number in randoms:
                        r = number
                        print(r)
                        random.seed(r)

                        for f in range(0, len(featureDF)):

                            featureSubset = featureDF[f] #pcaDF or featureDF[f]

                            targetClassCount = collections.Counter(featureSubset[featureSubset.columns[-1]])
                            majorityKey = max(targetClassCount, key=targetClassCount.get)
                            majorityCount = targetClassCount[majorityKey]
                            minorityKey = min(targetClassCount,  key=targetClassCount.get)
                            minorityCount = targetClassCount[minorityKey]

                            featureSubset_majority = featureSubset[featureSubset[featureSubset.columns[len(featureSubset.columns)-1]] == majorityKey]
                            featureSubset_minority = featureSubset[featureSubset[featureSubset.columns[len(featureSubset.columns)-1]] == minorityKey]
                            featureSubset_majority_downsampled = resample(featureSubset_majority, replace=False, n_samples=minorityCount, random_state=r) # reproducible results
                            featureSubset_downsampled = pd.concat([featureSubset_majority_downsampled, featureSubset_minority])
                            featureSubset_downsampled = featureSubset_downsampled.sample(frac=1).reset_index(drop=True)
                            featureSubset = featureSubset_downsampled

                            target = featureSubset[featureSubset.columns[-1]]  
                            featureSubset = featureSubset[featureSubset.columns[:-1]] #Skip PHQ-9 Responses 

                        #for i in range(0, len(modelList)):
                            featureList.append(nFeatureList[f])  ###########
                            flist.append(name)
                            mlist.append(modelType)
                            if modelType == "SVC1":
                                clf = svm.SVC(kernel='rbf', random_state=r)
                            elif modelType == "SVC2":
                                clf = svm.SVC(kernel='linear', random_state=r)
                            elif modelType == "RF":
                                clf = RandomForestClassifier(criterion="gini", max_depth=3, random_state=r)
                            elif modelType == "kNN3":
                                clf = KNeighborsClassifier(n_neighbors=3)
                            elif modelType == "kNN5":
                                clf = KNeighborsClassifier(n_neighbors=5)
                            elif modelType == "XG":
                                clf = xgb.XGBClassifier(max_depth=3, random_state=r)
                            elif modelType == "LR":
                                clf = LogisticRegression(random_state=r)
                            elif modelType == "NB":
                                clf = GaussianNB()

                            scores = cross_validate(clf, featureSubset, target, scoring=scoring,cv=5, return_train_score=False)
                            y_pred = cross_val_predict(clf, featureSubset, target, cv=5)

                            conf_mat = confusion_matrix(target, y_pred)
                            TN = conf_mat[0][0]
                            TP = conf_mat[1][1]
                            FP = conf_mat[0][1]
                            FN = conf_mat[1][0]
                            precision = mean(scores['test_precision'])
                            sensitivity = mean(scores['test_recall'])
                            f1 = mean(scores['test_f1'])
                            accuracy = mean(scores['test_accuracy'])
                            auc = mean(scores['test_roc_auc'])

                            precisionList.append(precision)
                            recallList.append(sensitivity)
                            f1List.append(f1)
                            accuracyList.append(accuracy)
                            aucList.append(auc)
                            truePosList.append(TP)
                            trueNegList.append(TN)
                            falsePosList.append(FP)
                            falseNegList.append(FN)
                            predictions.append(y_pred)
                            randomseed.append(r)
                            shape.append(df0.shape)

                        resultsDF = pd.DataFrame()
                        resultsDF["texts"] = flist
                        resultsDF["model"] = mlist
                        resultsDF["nFeatures"] = featureList
                        resultsDF["precision"] = precisionList
                        resultsDF["recall"] = recallList
                        resultsDF["F1"] = f1List
                        resultsDF["Accuracy"] = accuracyList
                        resultsDF["AUC"] = aucList
                        resultsDF["truePos"] = truePosList
                        resultsDF["trueNeg"] = trueNegList
                        resultsDF["falsePos"] = falsePosList
                        resultsDF["falseNeg"] = falseNegList
                        resultsDF["predictions"] = predictions
                        resultsDF["randomN"] = randomseed
                        resultsDF["shape"] = shape

                        resultsDF.to_csv("latencyResults/latencyResults" + name + str(split) + modelType + str(ndays) + "days" + featureE + ".csv")